# Install Dependencies

In [1]:
!pip install -U transformers
!pip install -q pillow accelerate einops
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install fastapi typing_extensions gradio==3.41.0

#Import Dependencies

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image
import gradio as gr

# Load model in GPU

In [3]:
torch.set_default_device("cuda")

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    "MILVLG/imp-v1-3b",
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(
    "MILVLG/imp-v1-3b",
    trust_remote_code=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
ImpModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/tran

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

#Prompt to the Model

In [5]:
text = """A chat between a curious user and an artificial intelligence assistant.
The assistant generates helpful and detailed answer after analyzing an image.
USER: <image>\nAnalyze the image.
ASSISTANT:"""

# Test case
#image = Image.open("sample_image.jpg")


In [7]:
"""
input_ids = tokenizer(text, return_tensors="pt").input_ids
image_tensor = model.image_preprocess(image)
output_ids = model.generate(
    input_ids,
    max_new_tokens=400,
    images=image_tensor,
    # Remove use_cache=True or set it to False
    use_cache=False  # or remove this line entirely
)[0]
print(tokenizer.decode(output_ids[input_ids.shape[1]:], skip_special_tokens=True).strip())
"""

The image displays a stock market chart with Dow Industrials as the ticker symbol. The chart shows the stock's performance over a period of time, with the price of the stock fluctuating. The stock is currently at a low point, as indicated by the downward trend in the chart. The chart also includes a red line, which could represent a significant drop in the stock's value. The overall impression is that the stock is experiencing a decline in its price.


#Generate response

In [12]:
def generate_answer(image):
  input_ids = tokenizer(text, return_tensors="pt").input_ids
  input_ids = input_ids.to('cuda')
  image_tensor = model.image_preprocess(image)
  output_ids = model.generate(
      input_ids,
      max_new_tokens=300,
      images=image_tensor,
      use_cache=False)[0]
  result = tokenizer.decode(output_ids[input_ids.shape[1]:], skip_special_tokens=True).strip()
  return result

# Deploy app with Gradio

In [13]:
iface = gr.Interface(fn=generate_answer,
                     inputs=gr.inputs.Image(type="pil"),
                     outputs="text",
                     title="Image Analyzer using a Small Vision Model",
                     description="Upload an image to analyze.")

# Run the app
iface.launch(share=True, debug=True)

<ipython-input-13-ec19370c2b9f>:2: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Image(type="pil"),
<ipython-input-13-ec19370c2b9f>:2: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Image(type="pil"),


IMPORTANT: You are using gradio version 3.41.0, however version 4.44.1 is available, please upgrade.
--------
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://6601e83b70e45677b2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6601e83b70e45677b2.gradio.live
